In [1]:
import pip
import sys
import signal
import time
import os
import re
from functools import reduce
from operator import iconcat
from dotenv import load_dotenv

import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()

load_dotenv()

True

In [2]:
# Package installer function to handle missing packages
def install(package):
    print(package + ' package for Python not found, pip installing now....')
    pip.main(['install', package])
    print(package + ' package has been successfully installed for Python\n Continuing Process...')

# Ensure beautifulsoup4 is installed
try:
    from bs4 import BeautifulSoup
except:
    install('beautifulsoup4')
    from bs4 import BeautifulSoup

# Ensure selenium is installed
try:
    from selenium import webdriver
except:
    install('selenium')
finally:
    from selenium import webdriver
    # from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException

In [53]:
class MarketWatchETL:
    def __init__(self):
        self.driver = None
        self._service = Service(
                '/Users/jerryli/Downloads/chromedriver-mac-arm64/chromedriver'
            )
        self._opts = webdriver.ChromeOptions()
        self._base_url = 'https://www.marketwatch.com/investing/stock/'
        self._retries = 10
        self._MW_USER = os.getenv('MARKETWATCH_USER')
        self._MW_KEY = os.getenv('MARKETWATCH_KEY')
        # self._service = Service('/Users/jerryli/Downloads/chromedriver-mac-arm64/chromedriver')
        # self.ticker = ticker.lower()
       # self._dcap = dict(DesiredCapabilities.PHANTOMJS)
        # self._opts = webdriver.ChromeOptions()
        # self._dcap = dict(DesiredCapabilities.CHROME)
        # self._dcap["phantomjs.page.settings.userAgent"] = ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        #                                                    "AppleWebKit/537.36 (KHTML, like Gecko) "
        #                                                    "Chrome/119.0.0.0 Safari/537.36")
        # self._base_url = 'https://www.marketwatch.com/investing/stock/'
        # self._retries = 10

    def login(self):
        self._initiate_driver()
        self.driver.get(self._base_url)
        
        close_popup_icon = self.driver.find_element(By.XPATH, "//div[@id='cx-scrim']//div[@id='cx-scrim-wrapper']/button")
        close_popup_icon.click()
        
        profile_xpath = "//div[@class='profile logged-out']/label[@class='btn--text btn--profile j-toggle-label']"
        profile_icon = self.driver.find_element(By.XPATH, profile_xpath)
        self.driver.execute_script("arguments[0].click();", profile_icon)
        
        options_xpath = "//ul[@class='profile__menu j-menu-contents j-toggle--profile']/li"
        option_list = self.driver.find_elements(By.XPATH, options_xpath)
        option_list[1].click()
        
        try:
            # username_xpath = "//div[@class='input-icon-container']/input[@id='username']"
            username_input_box = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, 'username'))
            )
            # driver.execute_script("arguments[0].click();", username_input_box)
            # username_input_box.clear()
            username_input_box.send_keys(self._MW_USER)
        except TimeoutException as e:
            print(e)
            
        continue_xpath = "//button[@class='solid-button continue-submit dj-btn-primary group']"
        continue_btn = self.driver.find_element(By.XPATH, continue_xpath)
        self.driver.execute_script("arguments[0].click();", continue_btn)
        # continue_btn.click()
        
        try:
            pw_xpath = "//div[@id='password-login-card-container']//div[@class='input-icon-container']/input[@id='password-login-password']"
            pw_class = "password dj-input w-full pr-10"
            pw_input_box = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, pw_xpath))
            )
            # self.driver.execute_script("arguments[0].click();", pw_input_box)
            # pw_input_box.clear()
            self.driver.execute_script(
                f"""document.getElementsByClassName("{pw_class}")[0].setAttribute('value', '{self._MW_KEY}')""",
                pw_input_box
            )
            # pw_input_box.send_keys(self._MW_KEY)
        except TimeoutException as e:
            print(e)
            
        signin_xpath = "//button[@class='solid-button new-design basic-login-submit dj-btn-primary group w-full']"
        signin_btn = WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, signin_xpath)))
        # signin_btn = self.driver.find_element(By.XPATH, signin_xpath)
        # self.driver.execute_script(
        #     """document.querySelector('button.solid-button.new-design.basic-login-submit.dj-btn-primary.group.w-full').click();"""
        #     )
        # self.driver.execute_script("arguments[0].click();", signin_btn)
        signin_btn.click()
        # self.driver.set_page_load_timeout(2)

    @staticmethod
    def _cleaned_key_data_object(ticker, raw_data):
        cleaned_data = {}
        raw_labels = raw_data['labels']
        raw_values = raw_data['values']
        i = 0
        for raw_label in raw_labels:
            raw_value = raw_values[i]
            cleaned_data.update({str(raw_label.get_text()): raw_value.get_text()})
            i += 1
        return {ticker: cleaned_data}
    
    @staticmethod
    def _cleaned_competitors_object(ticker, raw_data):
        cleaned_data = {}
        raw_names = raw_data['Name']
        raw_chgs = raw_data['Chg %']
        raw_mcs = raw_data['Market Cap']
        i = 0
        for raw_name in raw_names:
            raw_chg = raw_chgs[i]
            raw_mc = raw_mcs[i]
            cleaned_data.update({
                str(raw_name): {
                    'per_chg': raw_chg,
                    'market_cap': raw_mc
                }
            })
            i += 1
        return {ticker: cleaned_data}
    
    @staticmethod
    def _cleaned_financials_object(ticker, raw_data):
        cleaned_data = {}
        raw_item = raw_data['Item']
        raw_years = raw_data['Years']
        raw_values = raw_data['Values']
        
        item_unique = sorted(set(raw_item))
        year_unique = sorted(set(raw_years))

        for item in item_unique:
            year_val = {}
            for year in year_unique:
                year_val.update({year: raw_values['_'.join((item, year))]})
            cleaned_data.update({item: year_val})
        return {ticker: cleaned_data}

    def _initiate_driver(self):
        try:
            if self.driver is None:
                self.driver = webdriver.Chrome(
                    service=self._service, 
                    options=self._opts)
        except:
            print('***SETUP ERROR: The PhantomJS Web Driver is either not configured or incorrectly configured!***')
            sys.exit(1)
            
    def _get_site_url(self, ticker, **site_kwargs):
        _url = self._base_url + ticker
        if site_kwargs is not None:
            _sub_page = ''
            for k, v in site_kwargs.items():
                if 'page' in k:
                    _sub_page += '/' + str(v)
            _url += _sub_page
        return _url
        # self.driver.get(_url)
            
    def _scrape_key_data(self):
        raw_data_obj = {}
        labels, values = list(), list()
        i = 0
        while i < self._retries:
            try:
                # time.sleep(3)
                # html = self.driver.page_source
                html = self.driver.execute_script("return document.documentElement.outerHTML;")
                soup = BeautifulSoup(html, "html.parser")
                items = soup.find_all('li', class_="kv__item")
                for item in items:
                    labels.append(item.find('small', class_="label"))
                    values.append(item.find('span', class_="primary"))
                
                if labels and values:
                    raw_data_obj.update({'labels': labels})
                    raw_data_obj.update({'values': values})
                    break
                else:
                    i += 1
            except:
                i += 1
                continue
        if i == self._retries:
            print('Please check your internet connection!\nUnable to connect...')
            self.driver.service.process.send_signal(signal.SIGTERM)
            sys.exit(1)
        return raw_data_obj
    
    def _scrape_competitors(self):
        raw_data_obj = {}
        names, chgs, mktcap = list(), list(), list()
        i = 0
        while i < self._retries:
            try:
                # time.sleep(3)
                # html = self.driver.page_source
                html = self.driver.execute_script("return document.documentElement.outerHTML;")
                soup = BeautifulSoup(html, "html.parser")
                table = soup \
                            .find_all('div', class_="element element--table overflow--table Competitors")[0] \
                            .find('table')
                headers = table \
                            .find('thead') \
                            .find('tr', class_="table__row") \
                            .find_all('th')
                comp_name_lbl, per_chg_lbl, mkt_cap_lbl = [th.get_text() for th in headers]      
                rows = table \
                        .find('tbody') \
                        .find_all('tr')
                for td in rows:
                    comp_name, per_chg, mc = [text for text in td.get_text().split('\n') if text != '']
                    names.append(comp_name)
                    chgs.append(per_chg)
                    mktcap.append(mc)

                if names and chgs and mktcap:
                    raw_data_obj.update({comp_name_lbl: names})
                    raw_data_obj.update({per_chg_lbl: chgs})
                    raw_data_obj.update({mkt_cap_lbl: mktcap})
                    break
                else:
                    i += 1
            except:
                i += 1
                continue
        if i == self._retries:
            print('Please check your internet connection!\nUnable to connect...')
            self.driver.service.process.send_signal(signal.SIGTERM)
            sys.exit(1)
        return raw_data_obj

    def _scrape_financials(self, mode=None):
        def _fin_table_extraction(container, counter):
            _raw_data_obj = {}
            # table = soup \
            #             .find_all('div', class_="element element--table table--fixed financials")[0] \
            #             .find('table')
            table = container.find('table')
            headers = table \
                        .find('thead') \
                        .find('tr', class_="table__row") \
                        .find_all('th')
            _tmp_headers = [[_text for _text in th.get_text().split('\n') if _text != ''] for th in headers]
            item_lbl, _, *years_lbl = list(reduce(iconcat, _tmp_headers, []))
            rows = table \
                    .find('tbody') \
                    .find_all('tr')

            for td in rows:
                item_col_name, _, *item_vals = [text for text in td.get_text().split('\n') if text != '']
                
                assert len(years_lbl) == len(item_vals), 'Length of years ({}) is not the same as number of values ({})'.format(len(years_lbl), len(item_vals))
                for year, value in zip(years_lbl, item_vals):
                    item_col_name_list.append(item_col_name)
                    year_list.append(year)
                    year_val_dict.update({item_col_name + '_' + year: value})

            if item_col_name_list and year_list and year_val_dict:
                _raw_data_obj.update({item_lbl: item_col_name_list})
                _raw_data_obj.update({'Years': year_list})
                _raw_data_obj.update({'Values': year_val_dict})
            else:
                counter += 1
            return _raw_data_obj, counter
                
        temp_dict = {}
        year_list, item_col_name_list, year_val_dict = list(), list(), dict()
        i = 0
        # while i < self._retries:
        #     try:
        # time.sleep(2)
        # html = self.driver.page_source
        html = self.driver.execute_script("return document.documentElement.outerHTML;")
        soup = BeautifulSoup(html, "html.parser")
        table_container = soup.find_all('div', class_="element element--table table--fixed financials")
        
        if len(table_container) > 1:
            for container in table_container:
                raw_data_obj, i_new = _fin_table_extraction(container, i)
            if len(temp_dict) == 0:
                temp_dict.update(raw_data_obj)
            else:
                for k, v in temp_dict.items():
                    print(type(v))
                    if type(v) == list:
                        temp_dict[k] = v + raw_data_obj[k]
                    elif type(v) == dict:
                        temp_dict[k] = v | raw_data_obj[k]
                    
        else:
            temp_dict, i_new = _fin_table_extraction(table_container[0], i)
            
        # if i_new == i: break
            # except:
            #     i += 1
            #     continue
        # if i == self._retries:
            # print('Please check your internet connection!\nUnable to connect...')
            # self.driver.service.process.send_signal(signal.SIGTERM)
            # sys.exit(1)
        return temp_dict
        
    def get_competitor_data(self, ticker):
        url = self._get_site_url(ticker.lower())
        if self.driver.current_url != url:
            self.driver.get(url)
        _raw_comp_data = self._scrape_competitors()
        return self._cleaned_competitors_object(ticker.lower(), _raw_comp_data)
    
    def get_stock_key_data(self, ticker):
        url = self._get_site_url(ticker.lower())
        if self.driver.current_url != url:
            self.driver.get(url)
        _raw_key_data = self._scrape_key_data()
        return self._cleaned_key_data_object(ticker.lower(), _raw_key_data)
    
    def get_stock_financials(self, ticker, mode=None):
        pages = dict(page1='financials')
        match mode:
            case 'bs':
                pages |= dict(page2='balance-sheet')
            case 'bs_qt':
                pages |= dict(page2='balance-sheet', page3='quarter')
            case 'cf':
                pages |= dict(page2='cash-flow')
            case 'cf_qt':
                pages |= dict(page2='cash-flow', page3='quarter')
            case _:
                pass
        url = self._get_site_url(ticker.lower(), **pages)
        if self.driver.current_url != url:
            self.driver.get(url)
        _raw_comp_data = self._scrape_financials()
        return self._cleaned_financials_object(ticker.lower(), _raw_comp_data)

In [54]:
mw = MarketWatchETL()
mw.login()

In [63]:
msft_fin = mw.get_stock_financials('MSFT')
msft_fin_bs = mw.get_stock_financials('MSFT', mode='bs')
msft_fin_bs_qt = mw.get_stock_financials('MSFT', mode='bs_qt')
msft_fin_cf = mw.get_stock_financials('MSFT', mode='cf')
msft_fin_cf_qt = mw.get_stock_financials('MSFT', mode='cf_qt')
msft_comps = mw.get_competitor_data('MSFT')
msft_key_data = mw.get_stock_key_data('MSFT')

In [70]:
aapl_fin = mw.get_stock_financials('AAPL')
aapl_fin_bs = mw.get_stock_financials('AAPL', mode='bs')
aapl_fin_bs_qt = mw.get_stock_financials('AAPL', mode='bs_qt')
aapl_fin_cf = mw.get_stock_financials('AAPL', mode='cf')
aapl_fin_cf_qt = mw.get_stock_financials('AAPL', mode='cf_qt')
aapl_comps = mw.get_competitor_data('AAPL')
aapl_key_data = mw.get_stock_key_data('AAPL')

In [69]:
import json

tmp_filename = ['msft_fin', 'msft_fin_bs', 'msft_fin_bs_qt', 'msft_fin_cf', 'msft_fin_cf_qt', 'msft_comps', 'msft_key_data']
tmp_list = [msft_fin, msft_fin_bs, msft_fin_bs_qt, msft_fin_cf, msft_fin_cf_qt, msft_comps, msft_key_data]
for filename, tmp in zip(tmp_filename, tmp_list):
    with open(filename + '.json', 'w') as file:
        json.dump(tmp, file, indent=4)
        
tmp_filename = ['aapl_fin', 'aapl_fin_bs', 'aapl_fin_bs_qt', 'aapl_fin_cf', 'aapl_fin_cf_qt', 'aapl_comps', 'aapl_key_data']
tmp_list = [aapl_fin, aapl_fin_bs, aapl_fin_bs_qt, aapl_fin_cf, aapl_fin_cf_qt, aapl_comps, aapl_key_data]
for filename, tmp in zip(tmp_filename, tmp_list):
    with open(filename + '.json', 'w') as file:
        json.dump(tmp, file, indent=4)

In [72]:
mw.driver.quit()